## Setup

In [1]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

from os.path import exists
import pickle

## Import Data

In [2]:
holidays = pd.read_csv("../datasets/holidays_events.csv")
oil = pd.read_csv("../datasets/oil.csv")
stores = pd.read_csv("../datasets/stores.csv")
transactions = pd.read_csv("../datasets/transactions.csv") 

train = pd.read_csv("../datasets/train.csv")
test = pd.read_csv("../datasets/test.csv")


holidays["date"] = pd.to_datetime(holidays["date"])
oil["date"] = pd.to_datetime(oil["date"])
transactions["date"] = pd.to_datetime(transactions["date"])

train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])

## Data Transformations and Pipeline Building 

In [3]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [4]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [5]:
from sklearn.model_selection import train_test_split

train.drop(train[(train["date"] >= "01.04.2016") & (train["date"] < "01.05.2016")].index, axis="rows", inplace=True)

X_train, X_test, y_train, y_test = train_test_split(train.drop(["sales"], axis="columns"), train.loc[:, "sales"], test_size=0.1)

### Transactions Based Transformers

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class WeekdayTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X.loc[:, "date"] = pd.to_datetime(X["date"])
        X.loc[:, "is_Monday"] = X["date"].apply(lambda x: 1 if x.weekday() == 0 else 0)
        X.loc[:, "is_Thuesday"] = X["date"].apply(lambda x: 1 if x.weekday() == 1 else 0)
        X.loc[:, "is_Wednessday"] = X["date"].apply(lambda x: 1 if x.weekday() == 2 else 0)
        X.loc[:, "is_Thursday"] = X["date"].apply(lambda x: 1 if x.weekday() == 3 else 0)
        X.loc[:, "is_Friday"] = X["date"].apply(lambda x: 1 if x.weekday() == 4 else 0)
        X.loc[:, "is_Saturday"] = X["date"].apply(lambda x: 1 if x.weekday() == 5 else 0)
        X.loc[:, "is_Sunday"] = X["date"].apply(lambda x: 1 if x.weekday() == 6 else 0)
        return X


In [7]:
weekday_transformer = WeekdayTransformer()
weekday_transformer.fit_transform(X_train)

,id,date,store_nbr,family,onpromotion,is_Monday,is_Thuesday,is_Wednessday,is_Thursday,is_Friday,is_Saturday,is_Sunday
802195,802195,2014-03-28,17,SCHOOL AND OFFICE SUPPLIES,0,0,0,0,0,1,0,0
2553466,2553466,2016-12-06,54,PERSONAL CARE,6,0,1,0,0,0,0,0
745085,745085,2014-02-24,15,FROZEN FOODS,0,1,0,0,0,0,0,0
1079636,1079636,2014-08-30,51,DAIRY,3,0,0,0,0,0,1,0
991855,991855,2014-07-12,39,CLEANING,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2113114,2113114,2016-04-03,49,PERSONAL CARE,6,0,0,0,0,0,0,1
2024742,2024742,2016-02-14,2,PLAYERS AND ELECTRONICS,0,0,0,0,0,0,0,1
2410689,2410689,2016-09-17,49,CELEBRATION,0,0,0,0,0,0,1,0
24294,24294,2013-01-14,40,CELEBRATION,0,1,0,0,0,0,0,0


### Oil Based Transformers

In [8]:
class OilTransformer(BaseEstimator, TransformerMixin):

    def __init__ (self, oil_df):
        self.oil = oil_df

    def fit (self, X, y=None):
        oil_interpolated = pd.DataFrame()
        oil_interpolated["date"] = pd.date_range(start="2013-01-01", end="2017-08-31")
        oil_interpolated = oil_interpolated.merge(self.oil, on="date", how="left")
        oil_interpolated["dcoilwtico_interpolated"] = oil_interpolated["dcoilwtico"].interpolate()
        oil_interpolated["dcoilwtico_interpolated"] = oil_interpolated["dcoilwtico_interpolated"].bfill()

        self.oil = oil_interpolated

        return self

    def transform(self, X, y=None):
        X = X.merge(self.oil.loc[:, ["date", "dcoilwtico_interpolated"]], on="date")
        return X
        

In [9]:
oil_transformer = OilTransformer(oil)
oil_transformer.fit_transform(X_train)

,id,date,store_nbr,family,onpromotion,is_Monday,is_Thuesday,is_Wednessday,is_Thursday,is_Friday,is_Saturday,is_Sunday,dcoilwtico_interpolated
0,802195,2014-03-28,17,SCHOOL AND OFFICE SUPPLIES,0,0,0,0,0,1,0,0,101.730000
1,2553466,2016-12-06,54,PERSONAL CARE,6,0,1,0,0,0,0,0,50.950000
2,745085,2014-02-24,15,FROZEN FOODS,0,1,0,0,0,0,0,0,103.170000
3,1079636,2014-08-30,51,DAIRY,3,0,0,0,0,0,1,0,96.625000
4,991855,2014-07-12,39,CLEANING,0,0,0,0,0,0,1,0,101.563333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699190,2113114,2016-04-03,49,PERSONAL CARE,6,0,0,0,0,0,0,1,34.653333
2699191,2024742,2016-02-14,2,PLAYERS AND ELECTRONICS,0,0,0,0,0,0,0,1,29.185000
2699192,2410689,2016-09-17,49,CELEBRATION,0,0,0,0,0,0,1,0,43.140000
2699193,24294,2013-01-14,40,CELEBRATION,0,1,0,0,0,0,0,0,94.270000


### Stores Based Transformers

In [10]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

class StoresTransformer(BaseEstimator, TransformerMixin):

    def __init__ (self, stores_df):
        self.stores = stores_df

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.merge(stores, on="store_nbr")

        type_encoder = OrdinalEncoder()
        city_state_encoder = OneHotEncoder(sparse_output=False)

        X["type"] = type_encoder.fit_transform(X[["type"]])
        city_state_encoding_result = pd.DataFrame(city_state_encoder.fit_transform(X[["city", "state"]]), columns=city_state_encoder.get_feature_names_out())
        city_state_encoding_result = city_state_encoding_result.astype("bool")
        X = pd.concat([X, city_state_encoding_result], axis="columns")
        X = X.drop(["state", "city"], axis="columns")
        return X


In [11]:
stores_transformer = StoresTransformer(stores)
stores_transformer.fit_transform(X_train)

,id,date,store_nbr,family,onpromotion,is_Monday,is_Thuesday,is_Wednessday,is_Thursday,is_Friday,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,802195,2014-03-28,17,SCHOOL AND OFFICE SUPPLIES,0,0,0,0,0,1,...,False,False,False,False,False,False,True,False,False,False
1,2553466,2016-12-06,54,PERSONAL CARE,6,0,1,0,0,0,...,False,False,False,False,True,False,False,False,False,False
2,745085,2014-02-24,15,FROZEN FOODS,0,1,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
3,1079636,2014-08-30,51,DAIRY,3,0,0,0,0,0,...,True,False,False,False,False,False,False,False,False,False
4,991855,2014-07-12,39,CLEANING,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699190,2113114,2016-04-03,49,PERSONAL CARE,6,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
2699191,2024742,2016-02-14,2,PLAYERS AND ELECTRONICS,0,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
2699192,2410689,2016-09-17,49,CELEBRATION,0,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
2699193,24294,2013-01-14,40,CELEBRATION,0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False


### Holidays Based Transformer

In [12]:
from numpy import ndarray


class HolidaysTransformer(BaseEstimator, TransformerMixin):

    def __init__ (self, holidays):
        self.holidays = holidays

    @staticmethod
    def holidays_transformations(holidays:pd.DataFrame, df:pd.DataFrame):
        holidays.drop(holidays[holidays["transferred"] == True].index, inplace=True)
        work_days = holidays[holidays["type"] == "Work Day"]
        free_days = holidays[holidays["type"] != "Work Day"]  


        local_holidays = free_days[free_days["locale"] == "Local"]
        regional_holidays = free_days[free_days["locale"] == "Regional"]
        national_holidays = free_days[free_days["locale"] == "National"]

    
        df["is_Holiday"] = 0

        for idx in range(0, local_holidays.shape[0]):
            df.loc[(df["date"] == local_holidays.iloc[idx]["date"]) & 
                (df["city_" + local_holidays.iloc[idx]["locale_name"]] == local_holidays.iloc[idx]["locale_name"]), "is_Holiday"] = 1

        for idx in range(0, regional_holidays.shape[0]):
            df.loc[(df["date"] == regional_holidays.iloc[idx]["date"]) & 
                (df["state_" + regional_holidays.iloc[idx]["locale_name"]] == regional_holidays.iloc[idx]["locale_name"]), "is_Holiday"] = 1

        for idx in range(0, national_holidays.shape[0]):
            df.loc[df["date"] ==national_holidays.iloc[idx]["date"], "is_Holiday"]= 1

        return df
    

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = self.holidays_transformations(self.holidays, X)
        return X


### Custom Predictor

In [69]:
from sklearn.base import RegressorMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import check_is_fitted

class SplittedPredictor(RegressorMixin, BaseEstimator):

    def __init__ (self, n_estimators=100, criterion="squared_error", max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.families = None
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.models = {}
        self.store_without_items = None


    def __sklearn_is_fitted__(self):
        return hasattr(self, "_is_fitted") and self._is_fitted
        

    def split_dataframe(self, X:pd.DataFrame, y:pd.DataFrame=None) -> dict | tuple[dict] :        

        if y is None:
            splitted_dataframes = {}
            for family in self.families:
                splitted_dataframes[family] = X[X["family"] == family]
            return splitted_dataframes


        X_dropped = X.copy()
        
        for family, store_nbr in self.store_without_items:
            X_dropped.drop(X_dropped[(X_dropped["family"] == family) & (X_dropped["store_nbr"] == store_nbr)].index, axis="rows", inplace=True)

        splitted_dataframes_train = {}
        splitted_dataframes_target = {}

        for family in self.families :
            splitted_dataframes_train[family] = X_dropped[X_dropped["family"] == family]
            splitted_dataframes_target[family] = y.loc[X_dropped[X_dropped["family"] == family].index]

        return splitted_dataframes_train, splitted_dataframes_target

    def find_stores_without_items(self, X:pd.DataFrame, y:pd.DataFrame) -> list:
        target = pd.DataFrame({"target" : y})
        X = X.sort_index()
        temp = pd.concat([X, target], axis=1)
        aggregated_train = temp.groupby(["family", "store_nbr"])["target"].sum().reset_index()
        stores_without_items = aggregated_train[aggregated_train["target"] == 0].apply(lambda x : [x["family"], x["store_nbr"]], axis="columns").to_list()

        
        return stores_without_items
    

    def set_params(self, **params):
        if not params:
            return self

        for key, value in params.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                self.kwargs[key] = value


        if "families" in params:
            self.families = params["families"]
        
        if "store_without_items" in params:
            self.store_without_items = params["store_without_items"]

        self.models = {}
        self.store_without_items = None
        self.families = None
                
        return self


    def fit(self, X, y=None):

        X = X.sort_values(by="id")
        y = y.sort_index()
        self.families = list(X["family"].unique())
        for family in self.families:
            self.models[family] = RandomForestRegressor(n_estimators=self.n_estimators,
                                                        criterion=self.criterion, 
                                                        max_depth=self.max_depth,
                                                        min_samples_split=self.min_samples_split,
                                                        min_samples_leaf=self.min_samples_leaf)

        self.store_without_items = self.find_stores_without_items(X, y)
        splitted_dataframes_train, splitted_dataframes_target = self.split_dataframe(X, y)
        
        for family in self.families:
            if len(splitted_dataframes_train[family]) > 0:
                self.models[family].fit(splitted_dataframes_train[family].drop(["family"], axis="columns"), splitted_dataframes_target[family].to_numpy())
                print(f"Fit for {family} complited")



        self._is_fitted = True
        print("Fit complited")
        return self
            
    def predict(self, X):
        check_is_fitted(self)
        

        splitted_X = self.split_dataframe(X)
        splitted_predictions = {}

        result = None
        
        for family in self.families:
            if hasattr(self.models[family], 'estimators_') and len(self.models[family].estimators_) > 0:
                splitted_predictions[family] = self.models[family].predict(splitted_X[family].drop(["family"], axis="columns"))                
            else:
                splitted_predictions[family] = np.zeros(splitted_X[family].shape[0])    
            if result is None:
                result = pd.concat([splitted_X[family], pd.DataFrame( {"target" : splitted_predictions[family] }, index=splitted_X[family].index)], axis=1)
            else : result = pd.concat([result, pd.concat([splitted_X[family], pd.DataFrame( {"target" : splitted_predictions[family] }, index=splitted_X[family].index)] , axis=1)],  axis=0)

        for family, store_nbr in self.store_without_items:
            result.loc[((result["family"] == family) & (result["store_nbr"] == store_nbr)), "target"] = 0

        
        return result.sort_index().loc[:, "target"]

In [14]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
                ("weekdays", WeekdayTransformer()), 
                ("oil", OilTransformer(oil)),
                ("stores", StoresTransformer(stores)),
                ("holidays", HolidaysTransformer(holidays))
])

x = pipeline.fit_transform(pd.concat([X_train, y_train], axis=1))
x

,id,date,store_nbr,family,onpromotion,is_Monday,is_Thuesday,is_Wednessday,is_Thursday,is_Friday,...,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,is_Holiday
0,802195,2014-03-28,17,SCHOOL AND OFFICE SUPPLIES,0,0,0,0,0,1,...,False,False,False,False,False,True,False,False,False,0
1,2553466,2016-12-06,54,PERSONAL CARE,6,0,1,0,0,0,...,False,False,False,True,False,False,False,False,False,0
2,745085,2014-02-24,15,FROZEN FOODS,0,1,0,0,0,0,...,True,False,False,False,False,False,False,False,False,0
3,1079636,2014-08-30,51,DAIRY,3,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
4,991855,2014-07-12,39,CLEANING,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699190,2113114,2016-04-03,49,PERSONAL CARE,6,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,0
2699191,2024742,2016-02-14,2,PLAYERS AND ELECTRONICS,0,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,0
2699192,2410689,2016-09-17,49,CELEBRATION,0,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,0
2699193,24294,2013-01-14,40,CELEBRATION,0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0


## Hyperparametr tuning

In [91]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import r2_score

distributions = {
                 "n_estimators" : sp_randint(50, 1000), 
                 "criterion" : ["squared_error", "absolute_error", "friedman_mse", "poisson"], 
                 "max_depth" : sp_randint(10, 1000),
                 "min_samples_split" : sp_randint(2, 5),
                 "min_samples_leaf" : sp_randint(1, 5),
                 }

sc = SplittedPredictor()

ramdom_search = RandomizedSearchCV(sc, param_distributions=distributions, n_iter=1, cv=3, n_jobs=3)
ramdom_search

RandomizedSearchCV(cv=3, estimator=SplittedPredictor(), n_iter=1, n_jobs=3,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'friedman_mse',
                                                      'poisson'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEAE551390>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEB5C06550>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEAE52A710>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEAE314690>})

In [92]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(["sales"], axis="columns"), train.loc[:, "sales"], test_size=0.2)
x_test = pipeline.fit_transform(pd.concat([X_test, y_test], axis=1))
tempp = OrdinalEncoder()
x_test["family"] = tempp.fit_transform(x_test[["family"]]).astype(int)
x_test = x_test.set_index("id").sort_index()

In [93]:
ramdom_search.fit(x_test.drop(["sales", "date"], axis="columns"), x_test["sales"])   

Fit for 0 complited
Fit for 17 complited
Fit for 21 complited
Fit for 22 complited
Fit for 27 complited
Fit for 10 complited
Fit for 14 complited
Fit for 15 complited
Fit for 16 complited
Fit for 18 complited
Fit for 30 complited
Fit for 31 complited
Fit for 3 complited
Fit for 6 complited
Fit for 12 complited
Fit for 23 complited
Fit for 24 complited
Fit for 28 complited
Fit for 29 complited
Fit for 1 complited
Fit for 13 complited
Fit for 19 complited
Fit for 20 complited
Fit for 25 complited
Fit for 4 complited
Fit for 5 complited
Fit for 11 complited
Fit for 8 complited
Fit for 9 complited
Fit for 32 complited
Fit for 26 complited
Fit for 7 complited
Fit for 2 complited
Fit complited


RandomizedSearchCV(cv=3, estimator=SplittedPredictor(), n_iter=1, n_jobs=3,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'friedman_mse',
                                                      'poisson'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEAE551390>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEB5C06550>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEAE52A710>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001FEAE314690>})

In [94]:
ramdom_search.best_score_

0.7856423282905228

In [ ]:
######################################
# Unnecessary staff

In [84]:
#from sklearn.preprocessing import OrdinalEncoder
#
#X_train_pr = x.drop(["sales", "date"] , axis="columns")
#y_train_pr = x.loc[:, "sales"]
#temp = OrdinalEncoder()
#X_train_pr["family"] = temp.fit_transform(X_train_pr[["family"]])
#predator = SplittedPredictor()
#predator.fit(X_train_pr, y_train_pr)


In [85]:
#X_train_pr

In [86]:
#X_test_pr = pipeline.fit_transform(X_test).drop(["date"], axis="columns")
#X_test_pr["family"] = temp.transform(X_test_pr[["family"]])
#predictions = predator.predict(X_test_pr).sort_index()
#predictions

In [87]:
#predictions

In [88]:
#test_df = pd.read_csv("../datasets/test.csv")
#test_df["date"] = pd.to_datetime(test_df["date"])
#test_df["family"] = temp.transform(test_df[["family"]])
#predictions_test = predator.predict(pipeline.fit_transform(test_df).drop("date", axis="columns"))
#predictions_test


In [89]:
#predictions_test.to_csv("submit.csv") #score 0.49732